  \begin{equation}\label{eq:LVln}
    \begin{gathered}
      \dot{x}_{1}=x_{1}(3-x_{2}-x_{3}-x_{4})\,,\quad
      \dot{x}_{2}=x_{2}(x_{1}-x_{3})\,,\\
      \dot{x}_{3}=x_{3}(-1+x_{1}+x_{2}-x_{4})\,,\quad
      \dot{x}_{4}=x_{4}(-2+x_{1}+x_{3})\,,
    \end{gathered}
  \end{equation}
  $x_5=\ln(x_1), x_6=\ln(x_2), x_7=\ln(x_3), x_8=\ln(x_4)$

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import math
import os
import random
from sklearn.model_selection import train_test_split
output_directory = r'../../../Data/4D_Lotka_Volterra/4D_Lotka_T5_N40'
output_director = r'../../../System/4D_Lotka_Volterra/4D_Lotka_T5_N40'
m = 5  # number of trajectory
m1 = 21  # data point per trajectory divided by 2, which means the total data point is m*(2m1)=40
m2 = 20
def generate_random_values():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    return x1, x2, x3, x4
def generate_random_values_based_on_c():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    return x1, x2, x3, x4
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0:
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([x1 * (3 - x2 - x3 - x4), x2 * (x1 - x3), x3 * (-1 + x1 + x2 - x4), x4 * (-2 + x1 + x3)])
        normalized_f = normalize(f)
        return normalized_f
    t1 = np.linspace(0, 10, m1) # forward
    t2 = np.linspace(0, 10, m2) # backward
    forward_trajectories = []
    backward_trajectories = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4, x5):")
    for initial_condition in initial_conditions:
        print(initial_condition) 
        forward_trajectory = odeint(normalized_system, initial_condition, t1)
        backward_trajectory = odeint(normalized_system, initial_condition, -t2)
        forward_trajectories.append(forward_trajectory[:m1])
        backward_trajectories.append(backward_trajectory[:m2])
    return forward_trajectories, backward_trajectories, t1[:m1], t2[:m2]
def save_data(forward_trajectories, backward_trajectories, t1, t2):
    num_variables = 8 
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')    
    with open(os.path.join(output_director, '50.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, (forward_data, backward_data) in enumerate(zip(forward_trajectories, backward_trajectories)):
            for j in range(len(t1)):
                forward_row = forward_data[j].tolist()
                forward_row += [math.log(max(x, 1e-10)) for x in forward_row[:num_variables - 1]]
                forward_row += [r + 1]
                writer.writerow(forward_row)
            for j in range(1, len(t2)):  # Exclude initial conditions for backward trajectory
                backward_row = backward_data[j].tolist()
                backward_row += [math.log(max(x, 1e-10)) for x in backward_row[:num_variables - 1]]
                backward_row += [r + 1]
                writer.writerow(backward_row)  
    output_directory1 = r'../../../results/4D_Lotka_Volterra/4D_Lotka_T5_N40'
    plt.figure(figsize=(10, 6))
    for i, (forward_sol, backward_sol) in enumerate(zip(forward_trajectories, backward_trajectories)):
        for j in range(forward_sol.shape[1]):
            plt.plot(t1, forward_sol[:, j])
            plt.plot(-t2, backward_sol[:, j])  # Exclude initial conditions for backward trajectory
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i + 1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(m)]
    forward_trajectories, backward_trajectories, t1, t2 = generate_data(initial_conditions)
    save_data(forward_trajectories, backward_trajectories, t1, t2)
    data = np.genfromtxt(os.path.join(output_director, '50.csv'), delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4, x5):
(2.205388225869987, 1.2062278945128875, 1.5307491938736708, 2.6849314040666306)
(2.7460765262137317, 3.523211926739394, 0.694466261113357, 0.7189190895588493)
(2.8538016249298073, 1.12336102628843, 2.805268479911076, 3.9328444287631514)
(3.16319723693973, 1.1339493117089958, 1.802131201643952, 2.138595097797396)
(3.260958700058176, 1.242064692468134, 2.3907957659485546, 2.0215890762019892)


In [2]:
import pandas as pd
import os
input_directory = r'../../../System/4D_Lotka_Volterra/4D_Lotka_T5_N40'
output_directory = r'../../../Data/4D_Lotka_Volterra/4D_Lotka_T5_N40/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/4D_Lotka_Volterra/4D_Lotka_T5_N40/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)